## Tool Calling with the Converse API

This notebook will walk through how to use tool calling to complete agentic workflows.

In [ ]:
import boto3

LITE_MODEL_ID = "us.amazon.nova-lite-v1:0"

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1")

We'll create a basic weather tool that will generate a random temperature for the city passed in

The tools available to the model can be passed through the toolConfig parameter of the converse API. It is a best practice to be specific and consise. The schema will help the model determine when and how to use the tools available

In [ ]:
import string
import random


def get_weather(city: string):
    return f"{random.randint(0, 100)} degrees"


tool_config = {
    "tools": [
        {
            "toolSpec": {
                "name": "getWeather",  # Name of the tool
                "description": "A helpful weather tool to retrieve the current weather based on a city",  # Helpful description of the tool's functionality
                "inputSchema": {
                    "json": {
                        "type": "object",  # The top level schema MUST have a type of "object", properities and required keys. No other fields are allowed at this level
                        "properties": {
                            "city": {  # The name of the parameter
                                "type": "string",  # parameter type: string/int/etc
                                "description": "The city to retrieve the weather for.",  # Helpful description of the parameter
                            }
                        },
                        "required": ["city"],  # List of all required parameters
                    }
                },
            }
        }
    ]
}

Now that we've defined our tool, we'll put together some code that will call the model and invoke the tool if it is called.

Once the tool is called we pass it back to the model as a toolResult content type
```python
{ "role": "user", "content": [{ "toolResult": { "toolUseId": tool_use["toolUseId"], "content": [{ "json": { "weather": weather }}]}}]}
```

In [ ]:
import json


def nova_travel_guide(city: string):
    print("-----------------")
    user_message = f"Can you recommend some activities for me to do in {city} based on the current weather?"
    print(f"User Message: {user_message}")

    system = [
        {
            "text": "You are a helpful travel assistant. You have access to external tools to help give recommendations to the user"
        }
    ]

    messages = [{"role": "user", "content": [{"text": user_message}]}]

    inf_params = {"maxTokens": 300, "topP": 1, "temperature": 1}

    initial_response = client.converse(
        modelId=LITE_MODEL_ID,
        system=system,
        messages=messages,
        inferenceConfig=inf_params,
        additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
        toolConfig=tool_config,
    )
    print("\n[Initial Response]")
    print(f"Stop Reason: {initial_response['stopReason']}")
    print(f"Content: {json.dumps(initial_response['output']['message'], indent=2)}")

    if initial_response["stopReason"] == "tool_use":
        tool_use = next(
            block["toolUse"]
            for block in initial_response["output"]["message"]["content"]
            if "toolUse" in block
        )

        if tool_use["name"] == "getWeather":

            print(f"\nTool Name: {tool_use['name']}")
            print(f"Tool Input: {tool_use['input']}")

            weather = get_weather(tool_use["input"]["city"])

            print(f"Tool Result: {weather}")

            final_messages = [
                *messages,
                initial_response["output"]["message"],
                {
                    "role": "user",
                    "content": [
                        {
                            "toolResult": {
                                "toolUseId": tool_use["toolUseId"],
                                "content": [{"json": {"weather": weather}}],
                            }
                        }
                    ],
                },
            ]
            final_response = client.converse(
                modelId=LITE_MODEL_ID,
                messages=final_messages,
                inferenceConfig=inf_params,
                additionalModelRequestFields={"inferenceConfig": {"topK": 1}},
                toolConfig=tool_config,
            )

            output = next(
                block["text"]
                for block in final_response["output"]["message"]["content"]
                if "text" in block
            )
            print(f"\nResponse: {output}")
        else:
            print("The weather tool was not called")

Now lets interact with the tool!

In [ ]:
nova_travel_guide("San Fransisco")
nova_travel_guide("Houston")